In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

train_loader = DataLoader(dataset=mnist_train,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

test_loader = DataLoader(dataset=mnist_test,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [85]:
for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.long()

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 2.304
[1,   200] loss: 2.301
[1,   300] loss: 2.298
[1,   400] loss: 2.294
[1,   500] loss: 2.289
[1,   600] loss: 2.283
[2,   100] loss: 2.270
[2,   200] loss: 2.245
[2,   300] loss: 2.182
[2,   400] loss: 1.961
[2,   500] loss: 1.333
[2,   600] loss: 0.791
[3,   100] loss: 0.610
[3,   200] loss: 0.533
[3,   300] loss: 0.488
[3,   400] loss: 0.434
[3,   500] loss: 0.427
[3,   600] loss: 0.387
[4,   100] loss: 0.370
[4,   200] loss: 0.342
[4,   300] loss: 0.318
[4,   400] loss: 0.308
[4,   500] loss: 0.299
[4,   600] loss: 0.281
[5,   100] loss: 0.250
[5,   200] loss: 0.268
[5,   300] loss: 0.236
[5,   400] loss: 0.253
[5,   500] loss: 0.232
[5,   600] loss: 0.226
Finished Training


In [0]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [87]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))

GroundTruth:      2     4     2     2     3     3     8     3     2     9


In [88]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [0]:
outputs = net(images)

In [90]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(10)))

Predicted:      2     4     2     9     3     3     8     3     2     9


In [91]:
cnt = 0

for i in range(len(images)):
  if labels[i] == predicted[i]:
    cnt = cnt + 1

acc = cnt / len(images) * 100

print('ACC : ' + str(acc))

ACC : 95.0


In [92]:
dataiter = iter(test_loader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))

GroundTruth:      2     9     7     5     9     4     3     4     1     8


In [0]:
outputs = net(images)

In [94]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(10)))

Predicted:      2     9     2     5     9     0     3     4     1     8


In [95]:
cnt = 0

for i in range(len(images)):
  if labels[i] == predicted[i]:
    cnt = cnt + 1

acc = cnt / len(images) * 100

print('ACC : ' + str(acc))

ACC : 94.0


In [0]:
#custom dataset

In [96]:
import pickle
import cv2
import glob
import random
import numpy as np

image = []
label = []

for i in range(10):
    images1 = glob.glob('/gdrive/My Drive/mnist/' + str(i) + '/*.jpg')

    for fname1 in images1:
        print(fname1)
        image1 = cv2.imread(fname1, 0)
        image1 = cv2.resize(image1, (28, 28))
        image1.resize(1, 28, 28)
        image1 = cv2.bitwise_not(image1)
        image.append(image1)

        label1 = i
        label.append(label1)

with open('/gdrive/My Drive/pickle/data.p', 'wb') as f:
    pickle.dump(image, f, protocol=4)

with open('/gdrive/My Drive/pickle/label.p', 'wb') as f:
    pickle.dump(label, f, protocol=4)

custom_image = np.array(image)
custom_label = np.array(label)

print(custom_image.shape, custom_label.shape)

/gdrive/My Drive/mnist/0/001.jpg
/gdrive/My Drive/mnist/0/007.jpg
/gdrive/My Drive/mnist/0/000.jpg
/gdrive/My Drive/mnist/0/002.jpg
/gdrive/My Drive/mnist/0/004.jpg
/gdrive/My Drive/mnist/0/003.jpg
/gdrive/My Drive/mnist/0/005.jpg
/gdrive/My Drive/mnist/0/006.jpg
/gdrive/My Drive/mnist/0/009.jpg
/gdrive/My Drive/mnist/0/015.jpg
/gdrive/My Drive/mnist/0/010.jpg
/gdrive/My Drive/mnist/0/012.jpg
/gdrive/My Drive/mnist/0/013.jpg
/gdrive/My Drive/mnist/0/014.jpg
/gdrive/My Drive/mnist/0/011.jpg
/gdrive/My Drive/mnist/0/008.jpg
/gdrive/My Drive/mnist/0/016.jpg
/gdrive/My Drive/mnist/0/017.jpg
/gdrive/My Drive/mnist/0/018.jpg
/gdrive/My Drive/mnist/0/019.jpg
/gdrive/My Drive/mnist/0/025.jpg
/gdrive/My Drive/mnist/0/024.jpg
/gdrive/My Drive/mnist/0/022.jpg
/gdrive/My Drive/mnist/0/023.jpg
/gdrive/My Drive/mnist/0/026.jpg
/gdrive/My Drive/mnist/0/021.jpg
/gdrive/My Drive/mnist/0/027.jpg
/gdrive/My Drive/mnist/0/020.jpg
/gdrive/My Drive/mnist/0/028.jpg
/gdrive/My Drive/mnist/0/029.jpg
/gdrive/My

In [0]:
x_train  =  torch.FloatTensor(custom_image)
y_train  =  torch.IntTensor(custom_label)

In [0]:
custom_dataset = TensorDataset(x_train, y_train)

In [0]:
custom_dataloader = DataLoader(custom_dataset, batch_size=100, shuffle=True)

In [100]:
dataiter = iter(custom_dataloader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))

GroundTruth:      4     3     6     8     0     0     5     8     1     3


In [0]:
outputs = net(images)

In [102]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(10)))

Predicted:      9     3     5     5     9     0     5     8     6     2


In [103]:
cnt = 0

for i in range(len(images)):
  if labels[i] == predicted[i]:
    cnt = cnt + 1

acc = cnt / len(images) * 100

print('ACC : ' + str(acc))

ACC : 45.0
